In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import imageio
from IPython.display import Image

In [ ]:
RAW_DIR = "../data/brats20/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"  

In [ ]:
modalities = ['t1', 't1ce', 't2', 'flair']
volumes = {mod: nib.load(Path(RAW_DIR) / 'BraTS20_Training_001' / f'BraTS20_Training_001_{mod}.nii').get_fdata() for mod in modalities}


def show_slice(volume, slice_idx):
    """Display a single 2D slice from a 3D MRI volume."""
    plt.imshow(volume[:, :, slice_idx], cmap='gray')
    plt.axis('off')

# Pick the same slice index for all (usually middle)
mid_slice = volumes['t1'].shape[2] // 2

# Plot all 4 modalities in one figure
plt.figure(figsize=(12, 8))
for i, mod in enumerate(modalities):
    plt.subplot(2, 2, i + 1)
    show_slice(volumes[mod], mid_slice)
    plt.title(mod.upper())

plt.tight_layout()
plt.show()

# Utility: create a GIF from all slices in an MRI volume
def create_gif_from_volume(volume, output_path, axis=2, fps=10):
    """
    Create a GIF scrolling through the MRI slices.

    Args:
        volume (np.ndarray): 3D MRI data (x, y, z).
        output_path (Path): Path to save the GIF file.
        axis (int): Axis along which to slice (default=2 for axial view).
        fps (int): Frames per second of the GIF.
    """
    # Normalize volume to [0, 255] for visualization
    volume = (255 * (volume - np.min(volume)) / (np.ptp(volume))).astype(np.uint8)
    
    # Rearrange if needed
    if axis == 0:
        slices = [volume[i, :, :] for i in range(volume.shape[0])]
    elif axis == 1:
        slices = [volume[:, i, :] for i in range(volume.shape[1])]
    else:
        slices = [volume[:, :, i] for i in range(volume.shape[2])]
    
    # Save as GIF
    frames = [np.flipud(s) for s in slices]  # Flip vertically for correct orientation
    imageio.mimsave(output_path, frames, fps=fps)
    print(f"✅ GIF saved to {output_path}")
    
def numerical_explotation(volume):
    print(f"Shape: {volume.shape}")
    print(f"Min: {np.min(volume)}")
    print(f"Max: {np.max(volume)}")
    print(f"Mean: {np.mean(volume)}")
    print(f"Std: {np.std(volume)}")
    print(f"Non-zero voxels: {np.count_nonzero(volume)}")
    


## 🧠 MRI Modalities for Brain Tumor Analysis

MRI scans provide different **contrasts (modalities)** of the same brain, each revealing unique tissue properties.  
In datasets like **BraTS**, four modalities are used together to give a complete picture of anatomy, activity, and pathology.

| Modality | Main Use | Bright Areas | Dark Areas |
|-----------|-----------|---------------|-------------|
| **T1** | Structural baseline | White matter | CSF, edema, tumor |
| **T1CE** | Tumor enhancement (post-contrast) | Active tumor regions | CSF, necrosis |
| **T2** | Water & edema detection | CSF, edema | White matter |
| **FLAIR** | Lesions near CSF (CSF suppressed) | Edema, lesions | CSF |

```python
# Multimodal input (BraTS example)
t1, t1ce, t2, flair = load_modalities()
x = np.stack([t1, t1ce, t2, flair], axis=-1)  # (H, W, D, 4)


In [ ]:
show_slice(nib.load(Path(RAW_DIR) / "BraTS20_Training_001" / "BraTS20_Training_001_seg.nii").get_fdata(), 50)

subject_dir = Path(RAW_DIR)/ "BraTS20_Training_001"
flair_path = subject_dir / "BraTS20_Training_001_flair.nii"

if not flair_path.exists():
    raise FileNotFoundError(f"❌ File not found: {flair_path}")

# Load MRI volume
img = nib.load(flair_path)
volume = img.get_fdata()
output_gif = "../visualization/flair_animation.gif"
# create_gif_from_volume(volume, output_gif, axis=2, fps=8)
# Image(filename=output_gif)



In [ ]:
import nibabel as nib
import numpy as np
from pathlib import Path

# point directly to the folder containing patient directories
RAW_DIR = Path("../data/brats20/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData")
modalities = ["t1", "t1ce", "t2", "flair"]

for i in range(1, 6):  # iterate first 5 patients
    patient_id = f"BraTS20_Training_{i:03d}"
    patient_dir = RAW_DIR / patient_id

    print(f"\n=== Patient {patient_id} ===")

    for mod in modalities:
        path = patient_dir / f"{patient_id}_{mod}.nii"  # correct relative path
        if not path.exists():
            print(f"  ⚠️ Missing modality: {mod} at {path}")
            continue

        # Load volume
        volume = nib.load(path).get_fdata()

        # Compute stats excluding background (zeros)
        nonzero = volume[volume > 0]
        mean_val = np.mean(nonzero)
        std_val = np.std(nonzero)
        min_val = np.min(nonzero)
        max_val = np.max(nonzero)
        ratio_nonzero = 100 * len(nonzero) / volume.size

        # Print formatted summary
        print(f"  {mod.upper():<6} → shape: {volume.shape}, "
              f"min: {min_val:.1f}, max: {max_val:.1f}, "
              f"mean: {mean_val:.1f}, std: {std_val:.1f}, "
              f"nonzero: {ratio_nonzero:.2f}%")


## 🧠 Dataset Analysis Summary

### 🔍 Numerical Findings
After examining the first five **BraTS20** patient volumes across all four MRI modalities (`T1`, `T1CE`, `T2`, and `FLAIR`), several consistent patterns were observed:

| Observation | Explanation |
|--------------|-------------|
| **Constant spatial shape**: (240 × 240 × 155) | Confirms that the dataset is already spatially aligned and resampled to a common voxel grid. |
| **Dominant zero background (~83–86%)** | Indicates that a large portion of each volume corresponds to non-brain areas (air/background). These voxels carry no diagnostic value and would bias normalization if left in. |
| **Distinct modality intensity scales** | Each modality operates on a different numerical range: `T1` ≈ 0–800, `T1CE` ≈ 0–1800, `T2` and `FLAIR` ≈ 0–700–1000. Direct comparison requires normalization. |
| **Stable intra-modality statistics** | Means and standard deviations are relatively consistent within each modality, suggesting that simple per-volume normalization (e.g., z-score or min–max) should suffice. |

---

### 🧩 Interpretation
The results suggest that:
- No resampling is required — all volumes share identical dimensions.
- Intensity normalization is essential for cross-modality comparability.
- Explicit zero-masking is necessary before normalization to avoid corrupting the statistical distribution of the brain tissue.

By excluding zero voxels during normalization, we ensure that intensity scaling reflects true tissue properties instead of the background.

